In [1]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.model.rerank_model_finetune import RerankModelFinetune
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
bert=Bert_Extract(setting)
rerank_model_fintuned=RerankModelFinetune(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere_api,rerank_model_fintuned)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model


## 1.Eval Gemini model

In [2]:
# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')

In [3]:

pattern = r'Xin\s+lỗi\s+bạn'  # regex cho “Xin lỗi bạn” (có thể có nhiều khoảng trắng)

# Chỉ giữ lại những dòng không khớp pattern
df = df[~df['answer_from_gemini_rag_basic'].str.contains(pattern, regex=True, na=False)]

In [4]:
import time
def Generate_Response(df_batch: pd.DataFrame) -> list[str]:
    """
    Sinh câu trả lời cho mỗi câu hỏi trong batch.
    Trả về danh sách answer tương ứng.
    """
    answers = []
    for question in tqdm(df_batch['question'], desc="Sinh trả lời"):    
        article_doc = rag.get_gemini_response_rag_final(question)
        answers.append(article_doc)
        time.sleep(2)  # thử sleep 2-3 giây
    return answers

df['answer_from_gemini_rag_final'] = Generate_Response(df)


Sinh trả lời:   0%|          | 0/564 [00:00<?, ?it/s]

DEBUG - fixed_text:
 {
"answer": "Vấn đề ở đây là hành vi của bà B có dấu hiệu cưỡng ép kết hôn và vu khống người khác.\n\n*   Pháp luật nghiêm cấm hành vi cưỡng ép kết hôn (điểm b khoản 2 Điều 1).\n*   Người cưỡng ép người khác kết hôn trái với sự tự nguyện có thể bị xử lý hình sự, nếu trước đó đã bị xử phạt hành chính về hành vi này mà còn vi phạm (Điều 2).\n*   Hành vi vu khống có thể bị xử lý theo quy định của pháp luật hình sự.\n\nNhư vậy, việc làm của bà B có dấu hiệu vi phạm pháp luật hình sự về tội cưỡng ép kết hôn và tội vu khống.",
"key": [
2,
0
]
}


Sinh trả lời:   0%|          | 1/564 [01:16<11:55:23, 76.24s/it]

DEBUG - fixed_text:
 {
"answer": "Chào bạn, tôi hiểu rằng bạn đang muốn tìm hiểu về các biện pháp pháp lý để chấm dứt mối quan hệ ngoài hôn nhân của chồng bạn. Dưới đây là một số thông tin có thể hữu ích cho bạn:\n\n*   **Quyền yêu cầu ly hôn:** Bạn là vợ hợp pháp, do đó bạn có quyền yêu cầu Tòa án giải quyết ly hôn.\n*   **Thẩm quyền giải quyết ly hôn:** Tòa án sẽ là cơ quan có thẩm quyền giải quyết việc ly hôn của bạn.\n*   **Căn cứ ly hôn:** Việc chồng bạn chung sống như vợ chồng với người khác là một trong những căn cứ để bạn yêu cầu ly hôn.\n*   **Hậu quả pháp lý:** Tòa án sẽ xem xét các yếu tố như tài sản chung, quyền nuôi con để đưa ra phán quyết đảm bảo quyền lợi chính đáng của bạn và các con.\n\nĐể chấm dứt mối quan hệ sai trái này, bạn có thể làm đơn yêu cầu ly hôn gửi đến Tòa án có thẩm quyền. Tòa án sẽ giải quyết vụ việc theo quy định của pháp luật.\n",
"key": [
0
]
}


Sinh trả lời:   0%|          | 2/564 [02:29<11:39:49, 74.71s/it]

...

Sinh trả lời:  18%|█▊        | 99/564 [2:04:45<8:31:43, 66.03s/it]

DEBUG - fixed_text:
 {
"answer": "Chào bạn, tôi hiểu rằng bạn đang muốn biết về việc nhận con nuôi là cháu ruột và các điều kiện liên quan theo quy định của pháp luật Việt Nam. Dựa trên thông tin bạn cung cấp, tôi xin đưa ra một số thông tin như sau:\n\n*   **Về việc nhận con nuôi**: Pháp luật Việt Nam cho phép người thân trong gia đình nhận nhau làm con nuôi. Trong trường hợp của bạn, việc nhận cháu ruột làm con nuôi là có thể thực hiện được nếu đáp ứng các điều kiện theo quy định.\n\n*   **Hồ sơ cần nộp**: Bạn cần nộp hồ sơ của vợ chồng bạn (người nhận con nuôi) và hồ sơ của cháu bạn (người được giới thiệu làm con nuôi) tại Ủy ban nhân dân cấp xã nơi cháu bạn thường trú hoặc nơi vợ chồng bạn thường trú.\n\n*   **Thời gian giải quyết**: Ủy ban nhân dân cấp xã sẽ giải quyết hồ sơ trong thời hạn 30 ngày kể từ ngày nhận đủ hồ sơ hợp lệ.\n\n*   **Đăng ký nuôi con nuôi**: Nếu xét thấy cả hai bên (người nhận và người được nhận làm con nuôi) đều đủ điều kiện, Ủy ban nhân dân cấp xã sẽ tiến h

Sinh trả lời:  18%|█▊        | 100/564 [2:06:29<9:46:53, 75.89s/it]

DEBUG - fixed_text:
 {
"answer": "Tôi rất tiếc, tôi không thể cung cấp định nghĩa chính xác về \\"xâm hại tình dục trẻ em\\" vì không có thông tin nào trong tài liệu tham khảo định nghĩa thuật ngữ này. Tuy nhiên, tôi có thể cung cấp thông tin về các quy định của pháp luật liên quan đến hành vi xâm hại tình dục trẻ em:\n\n*   **Các hành vi bị coi là xâm hại tình dục và mức phạt:**\n    *   Giao cấu hoặc thực hiện hành vi quan hệ tình dục khác với người từ đủ 13 tuổi đến dưới 16 tuổi (không thuộc các trường hợp được quy định tại Điều 142 và Điều 144 của Bộ luật Hình sự) có thể bị phạt tù từ 01 năm đến 05 năm. Nếu phạm tội nhiều lần, với nhiều người, hoặc gây ra hậu quả nghiêm trọng hơn (như làm nạn nhân có thai, gây tổn hại sức khỏe), mức phạt có thể tăng lên từ 03 năm đến 10 năm.\n    *   Dùng vũ lực, đe dọa dùng vũ lực hoặc lợi dụng tình trạng không thể tự vệ của nạn nhân để giao cấu hoặc thực hiện hành vi quan hệ tình dục khác với người từ đủ 13 tuổi đến dưới 16 tuổi trái với ý muốn c

JSONDecodeError: Expecting ',' delimiter: line 2 column 76 (char 77)

In [5]:
text="""
 {
"answer": "Tôi rất tiếc, tôi không thể cung cấp định nghĩa chính xác về \\"xâm hại tình dục trẻ em\\" vì không có thông tin nào trong tài liệu tham khảo định nghĩa thuật ngữ này. Tuy nhiên, tôi có thể cung cấp thông tin về các quy định của pháp luật liên quan đến hành vi xâm hại tình dục trẻ em:\n\n*   **Các hành vi bị coi là xâm hại tình dục và mức phạt:**\n    *   Giao cấu hoặc thực hiện hành vi quan hệ tình dục khác với người từ đủ 13 tuổi đến dưới 16 tuổi (không thuộc các trường hợp được quy định tại Điều 142 và Điều 144 của Bộ luật Hình sự) có thể bị phạt tù từ 01 năm đến 05 năm. Nếu phạm tội nhiều lần, với nhiều người, hoặc gây ra hậu quả nghiêm trọng hơn (như làm nạn nhân có thai, gây tổn hại sức khỏe), mức phạt có thể tăng lên từ 03 năm đến 10 năm.\n    *   Dùng vũ lực, đe dọa dùng vũ lực hoặc lợi dụng tình trạng không thể tự vệ của nạn nhân để giao cấu hoặc thực hiện hành vi quan hệ tình dục khác với người từ đủ 13 tuổi đến dưới 16 tuổi trái với ý muốn của họ, hoặc giao cấu/quan hệ tình dục với người dưới 13 tuổi, có thể bị phạt tù từ 07 năm đến 15 năm. Các tình tiết tăng nặng (như loạn luân, làm nạn nhân có thai, gây tổn hại sức khỏe nghiêm trọng) có thể dẫn đến mức phạt từ 12 năm đến 20 năm.\n    *   Lợi dụng tình trạng lệ thuộc hoặc quẫn bách của người từ đủ 13 tuổi đến dưới 16 tuổi để khiến họ miễn cưỡng giao cấu hoặc thực hiện hành vi quan hệ tình dục khác có thể bị phạt tù từ 05 năm đến 10 năm. Các tình tiết tăng nặng tương tự như trên có thể dẫn đến mức phạt từ 07 năm đến 15 năm.\n*   **Các biện pháp bảo vệ nạn nhân:**\n    *   Khi có căn cứ xác định tính mạng, sức khỏe, tài sản, danh dự, nhân phẩm của người được bảo vệ (bao gồm cả nạn nhân) bị xâm hại hoặc bị đe dọa xâm hại do việc cung cấp chứng cứ, thông tin liên quan đến tội phạm, cơ quan có thẩm quyền có thể áp dụng các biện pháp bảo vệ như bố trí lực lượng bảo vệ, hạn chế đi lại/tiếp xúc, giữ bí mật thông tin, di chuyển chỗ ở, thay đổi lý lịch, răn đe/vô hiệu hóa hành vi xâm hại.\n    *   Các biện pháp bảo vệ an toàn cho nạn nhân và người thân thích của họ bao gồm bố trí nơi tạm lánh, giữ bí mật thông tin cá nhân, ngăn chặn hành vi xâm hại, và các biện pháp khác theo quy định của pháp luật.\n\nNhà nước có nhiều quy định để bảo vệ trẻ em khỏi bị xâm hại, bao gồm các điều khoản hình sự nghiêm khắc đối với hành vi xâm hại tình dục và các biện pháp bảo vệ nạn nhân và người thân của họ.",
"key": [
2,
0,
3,
1,
4
]
}"""


In [ ]:
import json
import re
import textwrap

def parse_raw_json(raw_text: str) -> dict:
    text = raw_text.replace('“', '"').replace('”', '"')
    pattern = r'("answer"\s*:\s*")(.+?)("(?=\s*,\s*"key"))'
    
    match = re.search(pattern, text, flags=re.DOTALL)
    if not match:
        print("DEBUG - raw_text:\n", raw_text)
        raise ValueError("Không tìm thấy trường 'answer' hoặc định dạng quá lệch không parse được.")
    
    prefix = match.group(1)
    answer_content = match.group(2)
    suffix = match.group(3)

    # Escape backslash, quotes và các ký tự xuống dòng trong answer_content
    escaped_content = (
        answer_content
        .replace('\\', '\\\\')
        .replace('"', '\\"')
        .replace('\n', '\\n')
        .replace('\r', '\\r')
        .replace('\t', '\\t')
    )

    fixed_text = (
        text[: match.start(1)] +
        prefix +
        escaped_content +
        suffix +
        text[match.end(3):]
    )

    fixed_text = textwrap.dedent(fixed_text).strip()
    return json.loads(fixed_text)


In [34]:
print(parse_raw_json(text)['answer'])

Tôi rất tiếc, tôi không thể cung cấp định nghĩa chính xác về \"xâm hại tình dục trẻ em\" vì không có thông tin nào trong tài liệu tham khảo định nghĩa thuật ngữ này. Tuy nhiên, tôi có thể cung cấp thông tin về các quy định của pháp luật liên quan đến hành vi xâm hại tình dục trẻ em:

*   **Các hành vi bị coi là xâm hại tình dục và mức phạt:**
    *   Giao cấu hoặc thực hiện hành vi quan hệ tình dục khác với người từ đủ 13 tuổi đến dưới 16 tuổi (không thuộc các trường hợp được quy định tại Điều 142 và Điều 144 của Bộ luật Hình sự) có thể bị phạt tù từ 01 năm đến 05 năm. Nếu phạm tội nhiều lần, với nhiều người, hoặc gây ra hậu quả nghiêm trọng hơn (như làm nạn nhân có thai, gây tổn hại sức khỏe), mức phạt có thể tăng lên từ 03 năm đến 10 năm.
    *   Dùng vũ lực, đe dọa dùng vũ lực hoặc lợi dụng tình trạng không thể tự vệ của nạn nhân để giao cấu hoặc thực hiện hành vi quan hệ tình dục khác với người từ đủ 13 tuổi đến dưới 16 tuổi trái với ý muốn của họ, hoặc giao cấu/quan hệ tình dục vớ

In [ ]:
final_output_path = './data/data_processed/final_data_system_response.csv'
df.to_csv(final_output_path, index=False)
print(f" Đã cập nhật và lưu toàn bộ vào: {final_output_path}")

> Configure

In [ ]:
setting=Settings()

> Get embedding gemini 

In [ ]:
import google.generativeai as genai

genai.configure(api_key=gemini.key_manager.get_next_key())

def get_gemini_embedding(texts):
    try :
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=texts,
            task_type="SEMANTIC_SIMILARITY"
        )
        
        return result['embedding']
    except Exception as e:
        print("Đã xảy ra lỗi:", e)
        return [0.0] * 768

> get embedding cohere 

In [ ]:
# import cohere

# def get_cohere_embedding(text: str, model_name: str = "embed-multilingual-v3.0") -> list:
#     try:
#         co = cohere.ClientV2(api_key=cohere_api.key_manager.get_next_key())
#         res = co.embed(
#             texts=[text],
#             model=model_name,
#             input_type="classification",
#             embedding_types=["float"],
#         )a

#         # Trả về nhúng đầu tiên
#         return res.embeddings.float[0]
#     except Exception as e:
#         print("Đã xảy ra lỗi:", e)
#         return [0.0] * 1024  # Trả về danh sách 1024 số 0 nếu có lỗi


>Get  rouge-L & F1 Score

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge_l(pred: str, ref: str) -> float:
    try:
        score = scorer.score(ref, pred)
        return score['rougeL'].fmeasure
    except:
        return 0.0

def compute_f1(pred: str, ref: str) -> float:
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not pred_tokens or not ref_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [ ]:
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')
df = df.dropna(subset=['answer', 'answer_from_gemini_rag_final'])

print(df.shape)

In [ ]:
# # df['answer_embedding_cohere'] = df['answer'].apply(get_cohere_embedding)
# df['answer_from_gemini_embedding_cohere'] = df['answer_from_gemini'].apply(get_cohere_embedding)


df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_gemini_rag_final_embedding'] = df['answer_from_gemini_rag_final'].apply(get_gemini_embedding)

df['cosine_similarity_gemini_rag_final'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']], 
        [row['answer_from_gemini_rag_final_embedding']]
    )[0][0],
    axis=1
)

# df['cosine_similarity_cohere'] = df.apply(
#     lambda row: cosine_similarity(
#         [row['answer_embedding_cohere']], 
#         [row['answer_from_gemini_embedding_cohere']]
#     )[0][0],
#     axis=1
# )

In [ ]:
df['rouge_l'] = df.apply(lambda row: compute_rouge_l(row['answer_from_gemini_rag_final'], row['answer']), axis=1)
df['f1'] = df.apply(lambda row: compute_f1(row['answer_from_gemini_rag_final'], row['answer']), axis=1)


In [ ]:
print(f" Trung bình Cosine (Gemini): {df['cosine_similarity_gemini_rag_final'].mean():.4f}")
print(f" Trung bình ROUGE-L: {df['rouge_l'].mean():.4f}")
print(f" Trung bình F1 Score: {df['f1'].mean():.4f}") 